1. Model and Data decisions:

 a. What data source did you choose and why?
- I chose the data source of stackoverflow, which is a stck of all programming related quesiton and asnwers. It is whre people share their errors they are facing while coding and get guidacne by the community. By training the LLM model with this data, we are training the model the can offer relavant ansers to the programming questions asked by the users making it easier for them.
  
  
 b. What business/environmental/social/political problem will your fine-tuned model tackle?
 
- The model will overcome the challenge of quicly finding the answers from the sea of programming questions. Where nowadays, the solving problem on time is crucial, this will help aid the process and bring effeciency in software developmnt. It will help troubleshoot the problems swiftly and accurately. 

2.  Explain to me, in your own words

 a. What is LoRA, and why is it necessary?
 
- LoRA, organizes the amssive chunks of information that a model uses to make decisons,in a way that lets the model update and adjsut itself without doing much redos.  It works by freezing the ordiginal model weights and decmposing the original matrix into two. With this, training is faster, and effecient in memory.
 
 b. What is QLoRA and why is it necessary?
 
- QLoRA  is an extension of LoRA that quantized the model to reduce the precision of model parameters. It is necessary because it significnatly lowers the computational cost while also preseving the eprformance of the model.
 
 c. What is instruction fine-tuning and how is it different from the unsupervised
 pre-training?
 
 - Instrcution fine tuning is teaching the model to perform certain tasks in a certain way , like instructing it by giving direct commands.  While, unsupervised learning is not supervising the model and letting it explore on itw own without any direct commands.
 
 d. Is what you are doing here considered instruction fine-tuning or unsupervised
 pre-training? why
 
 - What we are doing here is considered instruction fine tuning, because we are taking a pre-trained model and adapting that model to the tasks, which is fine-tuning, we are not training the model from scratch which would be unsupervised training.

In [ ]:
from dataclasses import dataclass, field
import os
from typing import Optional

@dataclass
class ScriptArguments:

    hf_token: str = field(metadata={"help": "Authentication Token for hugging face"})


    model_name: Optional[str] = field(
        default="meta-llama/Llama-2-7b-hf", metadata={"help": "The model name on the hugging face that we are using"}
    )

    seed: Optional[int] = field(
        default=4761, metadata = {'help':'Random seed to intialize'}
    )

    data_path: Optional[str] = field(
        default="./data/forums_short.json", metadata={"help": "The path of the data that we are using to fine tune the llm"}
    )

    output_dir: Optional[str] = field(
        default="output", metadata={"help": "the path of the output directory where we want to save the fine-tuned outputs"}
    )
    
    per_device_train_batch_size: Optional[int] = field(
        default = 2, metadata = {"help":"Numbe rof training samples processed on each GPU per step"}
    )

    gradient_accumulation_steps: Optional[int] = field(
        default = 1, metadata = {"help":"Numbe rof training steps to accumulate gradient before a backward/upward pass "}
    )

    optim: Optional[str] = field(
        default = "paged_adamw_32bit", metadata = {"help":"Selecting which optimizer to usefor the fine tuning"}
    )

    save_steps: Optional[int] = field(
        default = 25, metadata = {"help":"Number of trianing steps before saving the progress on the model"}
    )

    logging_steps: Optional[int] = field(
        default = 1, metadata = {"help":"How often to get progres sof the model's training"}
    )

    learning_rate: Optional[float] = field(
        default = 2e-4, metadata = {"help":"The initisl rate at which the model will learn,"}
    )

    max_grad_norm: Optional[float] = field (
        default = 0.3, metadata = {"help":"Max threshold to prevent the model's learning to go off track"}
    )

    num_train_epochs: Optional[int] = field (
        default = 1, metadata = {"help":"Number of times the training dataset will go throughthe entire dataset"}
    ) 

    warmup_ratio: Optional[float] = field (
        default = 0.03, metadata = {"help":"Numbe rof steps used for a linear warmup of the training model"}
    )

    lr_scheduler_type: Optional[str] = field(
        default="cosine", metadata = {"help":"Adjusts the learning rate as training progresses, how model learns over time"}
    ) 

    lora_dir: Optional[str] = field(default = "./model/llm_hate_speech_lora", metadata = {"help":"Directory where LoRA is saved"})

    max_steps: Optional[int] = field(default=-1, metadata={"help": "The total number of training set to be perofrmed, if positive"})

    text_field: Optional[str] = field(default='chat_sample', metadata={"help": "Default name of the data field that contains the text that has been used to train the model"})


In [ ]:
import os
import transformers

#Access tools from hugging face's library 
from transformers import (
    AutoModelForCausalLM, 
    AutoTokenizer,
    set_seed,
    BitsAndBytesConfig,
    Trainer,
    TrainingArguments,
    HfArgumentParser
)
from datasets import load_dataset
import torch
import bitsandbytes as bnb
from huggingface_hub import login, HfFolder
from trl import SFTTrainer
from utils import print_trainable_parameters, find_all_linear_names
from train_args import ScriptArguments
from peft import LoraConfig, get_peft_model, PeftConfig, PeftModel, prepare_model_for_kbit_training

#Setting up a parser to read and write cmd line args
parser = HfArgumentParser(ScriptArguments)
args = parser.parse_args_into_dataclasses()[0]

#Training the model with the hyper-paremeters initalized above
def training_function(args):
    
    login(token=args.hf_token) #login using the authenticaion token

    set_seed(args.seed) #Setting a random seed

    data_path=args.data_path #path to the training data

    dataset = load_dataset(data_path) #loading the dataset with the given path
    
    #Configuring bits and bytes
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16,
    )

    #Loading a pre-trained model
    model = AutoModelForCausalLM.from_pretrained(
        args.model_name,
        use_cache=False,
        device_map="auto",
        quantization_config=bnb_config,
        trust_remote_code=True
    )

    #Loading the tokenizer and setting the padding token and dorection 
    tokenizer = AutoTokenizer.from_pretrained(args.model_name)
    tokenizer.pad_token=tokenizer.eos_token
    tokenizer.padding_side='right'
    
    #Prepare the model for k-bit training
    model=prepare_model_for_kbit_training(model)

    #Finding all linear modules within the model
    modules=find_all_linear_names(model)
    #LoRA cinfiguration
    config = LoraConfig(
        r=64,
        lora_alpha=16,
        lora_dropout=0.1,
        bias='none',
        task_type='CAUSAL_LM',
        target_modules=modules
    )

    #Fetchig model with LoRA adjustments
    model=get_peft_model(model, config)
    output_dir = args.output_dir #Directory to save output
    
    #Passing all the training arguments for the model
    training_arguments = TrainingArguments(
        output_dir=output_dir,
        per_device_train_batch_size=args.per_device_train_batch_size,
        gradient_accumulation_steps=args.gradient_accumulation_steps,
        optim=args.optim,
        save_steps=args.save_steps,
        logging_steps=args.logging_steps,
        learning_rate=args.learning_rate,
        bf16=False,
        max_grad_norm=args.max_grad_norm,
        num_train_epochs=args.num_train_epochs,
        warmup_ratio=args.warmup_ratio,
        group_by_length=True,
        lr_scheduler_type=args.lr_scheduler_type,
        tf32=False,
        report_to="none",
        push_to_hub=False,
        max_steps = args.max_steps
    )
    #Initlaizgin trainer for sparse fine tuning
    trainer = SFTTrainer(
        model=model,
        train_dataset=dataset['train'],
        dataset_text_field=args.text_field,
        max_seq_length=2048,
        tokenizer=tokenizer,
        args=training_arguments
    )
    
    #Setting data nomrlaizers to use 32-bit precision
    for name, module in trainer.model.named_modules():
        if "norm" in name:
            module = module.to(torch.float32)

    print('starting training')

    #Startnig the training
    trainer.train()

    print('LoRA training complete')
    lora_dir = args.lora_dir #Directory to save the LoRA adapted model
    trainer.model.push_to_hub(lora_dir, safe_serialization=False) #Pushing the model to Hugging face hub
    
    print("saved lora adapters")
    
if __name__=='__main__':
    training_function(args) #Start the training if the script is executed 

In [ ]:
import os

import transformers
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    set_seed,
    BitsAndBytesConfig,
    Trainer,
    TrainingArguments,
)
from datasets import load_from_disk
import torch

import bitsandbytes as bnb
from huggingface_hub import login, HfFolder

import torch.nn as nn

# COPIED FROM https://github.com/artidoro/qlora/blob/main/qlora.py
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        num_params = param.numel()
        # if using DS Zero 3 and the weights are initialized empty
        if num_params == 0 and hasattr(param, "ds_numel"):
            num_params = param.ds_numel

        all_param += num_params
        if param.requires_grad:
            trainable_params += num_params
    trainable_params /= 2
    print(
        f"all params: {all_param:,d} || trainable params: {trainable_params:,d} || trainable%: {100 * trainable_params / all_param}"
    )

#Formatting a prommpt to ask the model
def format_prompt(eval_str):
    prompt=f"""###User: Categorize the following statement as offensive, neutral, or hateful: \n\n'{eval_str}'\n
follow this format with your response:

(1) Holistic Explanation: Provide a brief explanation (up to 150 words) on why the statement falls into the chosen category. If neutral, explain why it's neutral. If it's offensive or hateful, summarize the harmful impact,  detailing its impact on each group mentioned.

Classification: (neutral, offensive, hate)

If the statement is categorized as offensive or hateful, also include the following:

(2) Affected Groups: List the group(s) most harmed by the statement.
(3) Harmful Words: Identify specific harmful word(s) or phrase(s) directed at each group.
(4) Reason for Harm: Briefly explain why the words are hurtful to each group.

### Assistant:
"""
    return prompt

#Neural network class with 1 hidden layer and an output layer, with ReLU activation in between
class Multiclass(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(384, 8)
        self.act = nn.ReLU()
        self.output = nn.Linear(8, 3)
     #Forward pass of nueral net   
    def forward(self, x):
        x = self.act(self.hidden(x))
        x = self.output(x)
        return x


# COPIED FROM https://github.com/artidoro/qlora/blob/main/qlora.py
# Fiding and listing all the names of linear modules in amodel
def find_all_linear_names(model):
    lora_module_names = set()
    for name, module in model.named_modules():
        if isinstance(module, bnb.nn.Linear4bit):
            names = name.split(".")
            lora_module_names.add(names[0] if len(names) == 1 else names[-1])

    if "lm_head" in lora_module_names:  # needed for 16-bit
        lora_module_names.remove("lm_head")
    return list(lora_module_names)
